In [1]:
import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
from torch.utils.data import ConcatDataset

### Initialization & Preprocessing

In [2]:
IMG_HEIGHT = 48
IMG_WIDTH = 48

# Path to the training data
TRAIN_DATA_PATH = os.path.join(os.getcwd(), 'data', 'train')

# Path to the test data
TEST_DATA_PATH = os.path.join(os.getcwd(), 'data', 'test')

# Disgust Samples Multiplier
MULTIPLIER = 3

# Define your transformations
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor()
])

# Load the datasets
train_dataset = ImageFolder(TRAIN_DATA_PATH, transform=transform)
test_dataset = ImageFolder(TEST_DATA_PATH, transform=transform)

# Create the dataloader for validation set only, train data still needs to be augmented
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Oversampling the disgust samples since we don't have many samples

# Define additional transformations for data augmentation
augment_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Create a new dataset with only the "disgust" images
disgust_dataset = [img for img in train_dataset if img[1]
                   == train_dataset.class_to_idx['disgust']]

# Convert Tensor to PIL Image
to_pil = ToPILImage()

# Apply data augmentation to the "disgust" images
augmented_disgust_dataset = [(augment_transform(to_pil(img[0])), img[1])
                             for _ in range(MULTIPLIER) for img in disgust_dataset]

# Combine the original dataset with the augmented "disgust" images
train_dataset = ConcatDataset([train_dataset, augmented_disgust_dataset])

# Update the train DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Count the number of "disgust" samples in the train_dataset
num_disgust_samples = len(disgust_dataset)

print(f"Number of 'disgust' samples: {num_disgust_samples * MULTIPLIER}")

Number of 'disgust' samples: 1308


In [3]:
print(torch.cuda.is_available())

False


### Train and Evaluate

In [ ]:
import torch
from torch import optim, nn
from cnn import FERModel
from tqdm import tqdm
from sklearn.metrics import classification_report
import numpy as np

# Check if GPU is available and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# Create an instance of the model
num_classes = 7  # replace with the number of classes in your dataset
model = FERModel(num_classes)

# Move the model to the GPU if available
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs to train for
num_epochs = 5

# Loop over the dataset multiple times
for epoch in range(num_epochs):
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=int(len(train_loader)))
    for i, data in progress_bar:
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        progress_bar.set_description(
            f"Epoch {epoch + 1} loss: {running_loss/(i+1)}")

# Save the trained model
torch.save(model.state_dict(), 'model_high.pth')

# Evaluate the model on the test set and print a classification report
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

print(classification_report(all_labels, all_predictions))

print('Finished Training')

In [ ]:
for dataset in train_dataset.datasets:
    try:
        print(dataset.class_to_idx)
    except AttributeError:
        print("")
